In [9]:

#https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/03/08/image-mean-std.html
import numpy as np
import pandas as pd

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import os


####### PARAMS

device = torch.device('cuda')
num_workers = 8
image_size = 1024
batch_size = 8
data_path = 'PESMOD/train/images'


class LeafData(Dataset):

    def __init__(self,
                 data,
                 directory,
                 transform=None):
        self.data = data
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        # import
        path = os.path.join(self.directory, self.data[idx])
        image = cv2.imread(path)

        # augmentations
        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image


augs = A.Compose([A.Resize(height=image_size,
                           width=image_size),
                  #A.Normalize(mean=(0, 0, 0),
                  #            std=(1, 1, 1)),
                  ToTensorV2()])

files = list(sorted(os.listdir(data_path)))

image_dataset = LeafData(data=files,
                         directory=data_path,
                         transform=augs)

# data loader
image_loader = DataLoader(image_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=num_workers,
                          pin_memory=True)

# display images
for batch_idx, inputs in enumerate(image_loader):
    fig = plt.figure(figsize=(14, 7))
    for i in range(8):
        ax = fig.add_subplot(2, 4, i + 1, xticks=[], yticks=[])
        plt.imshow(inputs[i].numpy().transpose(1, 2, 0))
    break

####### COMPUTE MEAN / STD

# placeholders
psum = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader):
    psum += inputs.sum(axis=[0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis=[0, 2, 3])
# pixel count
count = len(files) * image_size * image_size

# mean and std
total_mean = psum / count
total_var = (psum_sq / count) - (total_mean ** 2)
total_std = torch.sqrt(total_var)

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))


 38%|████████████████████████████████████████████▋                                                                       | 164/426 [00:48<01:16,  3.40it/s]